In [1]:
from astropy.io import fits
from astropy.timeseries import TimeSeries, LombScargle
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import savgol_filter
import pandas as pd
import glob
from astroquery.mast import Tesscut
from astropy.coordinates import SkyCoord
from astroquery.mast import Observations
import astropy.units as u
import lightkurve as lk

c:\Users\Pablo\OneDrive\School of life\Coding\Codigo\Python\practice1\.venv\Lib\site-packages\lightkurve\prf\__init__.py:7: UserWarning: Warning: the tpfmodel submodule is not available without oktopus installed, which requires a current version of autograd. See #1452 for details.
  warnings.warn(


In [2]:
# search = lk.search_lightcurve("TIC 307210830", mission="TESS")
# print(search)

# lc_collection = search.download_all(download_dir="../data/")
# combined_lc = lc_collection.stitch().normalize()
# combined_lc.plot()


In [3]:
# Collect all FITS files in both HLSP and TESS folders
fits_files = glob.glob("../data/mastDownload/**/**/*.fits", recursive=True)
print(f"Found {len(fits_files)} FITS files")

Found 273 FITS files


In [4]:

def period_features(time, flux):
    try:
        ls = LombScargle(time, flux)
        freq, power = ls.autopower()
        best_period = 1 / freq[np.argmax(power)]
        best_power = np.max(power)
        return best_period, best_power
    except Exception:
        return 0, 0


In [5]:
def shape_features(local_flux):
    if len(local_flux) < 3:
        return 0, 0, 0
    depth = 1.0 - np.min(local_flux)
    width = np.sum(local_flux < (1 - depth/2))  # number of points below half-depth
    # symmetry: compare first half vs second half of dip
    symmetry = np.corrcoef(local_flux[:len(local_flux)//2],
                            local_flux[len(local_flux)//2:])[0, 1] if len(local_flux) > 4 else 0
    return depth, width, symmetry

In [6]:
def extract_features_from_file(path, threshold_percentile=5):  # Changed to 5% threshold
    try:
        hdul = fits.open(path)
        data = hdul[1].data
        time = data['TIME']
        flux = data['PDCSAP_FLUX']
        quality = data['QUALITY']

        # Mask
        mask = (~np.isnan(time)) & (~np.isnan(flux)) & (quality == 0)
        time_clean, flux_clean = time[mask], flux[mask]
        if len(time_clean) == 0:
            return [], []

        flux_norm = flux_clean / np.median(flux_clean)
        
        # Calculate period features once for the entire lightcurve
        best_period, best_power = period_features(time_clean, flux_norm)
        
        # Find dips using percentile threshold - increased to 5%
        threshold = np.percentile(flux_norm, threshold_percentile)
        dip_mask = flux_norm < threshold
        dip_indices = np.where(dip_mask)[0]
        
        features, labels = [], []
        
        # Process transit candidates (dips)
        for idx in dip_indices:
            # Extract a window around the dip
            window_start = max(0, idx-20)
            window_end = min(len(flux_norm), idx+20)
            local_flux = flux_norm[window_start:window_end]
            local_time = time_clean[window_start:window_end]
            
            # Calculate shape features
            depth, width, symmetry = shape_features(local_flux)
            
            # Calculate additional features
            local_slope = np.polyfit(range(len(local_flux)), local_flux, 1)[0]
            local_std = np.std(local_flux)
            window_ratio = np.mean(local_flux) / np.mean(flux_norm)
            snr = depth / (local_std + 1e-6)
            
            # Relaxed criteria for transit identification
            is_likely_transit = (width > 2 and 
                               symmetry > 0.3 and  # Relaxed from 0.6
                               snr > 3 and         # Relaxed from 7
                               local_slope < 0.02) # Relaxed from 0.01            
            features.append([
                local_std, width, symmetry, local_slope,
                window_ratio, snr, best_power
            ])
            labels.append(1 if is_likely_transit else 0)
        
        # Add non-transit baseline samples
        if len(dip_indices) > 0:
            non_dip_mask = ~dip_mask
            normal_indices = np.random.choice(
                np.where(non_dip_mask)[0],
                min(len(dip_indices), sum(non_dip_mask)),
                replace=False
            )
            
            for idx in normal_indices:
                window_start = max(0, idx-20)
                window_end = min(len(flux_norm), idx+20)
                local_flux = flux_norm[window_start:window_end]
                
                depth, width, symmetry = shape_features(local_flux)
                local_std = np.std(local_flux)
                local_slope = np.polyfit(range(len(local_flux)), local_flux, 1)[0]
                window_ratio = np.mean(local_flux) / np.mean(flux_norm)
                snr = depth / (local_std + 1e-6)
                
                features.append([
                    local_std, width, symmetry, local_slope,
                    window_ratio, snr, best_power
                ])
                labels.append(0)  # Definitely not a transit
                
        # Add source file as metadata
        return features, labels, [path] * len(features)

    except Exception as e:
        print(f"Error in {path}: {e}")
        return [], [], []

In [ ]:
all_features, all_labels, all_sources = [], [], []
for path in glob.glob("../data/**/**/*.fits", recursive=True):
    feats, labels, sources = extract_features_from_file(path, threshold_percentile=5)  # Changed to 5% threshold
    all_features.extend(feats)
    all_labels.extend(labels)
    all_sources.extend(sources)
print(f"Processed FITS files: {len(glob.glob('../data/**/**/*.fits', recursive=True))}")

df = pd.DataFrame(all_features, columns=[
    "local_noise", "width", "symmetry", "local_slope",
    "window_ratio", "snr", "best_power"
])
df["label"] = all_labels
df["source_file"] = all_sources

# Print label distribution to verify we have both classes
print("Label distribution:", df.label.value_counts())

Error in ../data\kplr011446443-2009131110544_slc.fits: "Key 'QUALITY' does not exist."
Error in ../data\mastDownload\HLSP\hlsp_qlp_tess_ffi_s0002-0000000307210830_tess_v01_llc\hlsp_qlp_tess_ffi_s0002-0000000307210830_tess_v01_llc.fits: "Key 'PDCSAP_FLUX' does not exist."
Error in ../data\mastDownload\HLSP\hlsp_qlp_tess_ffi_s0005-0000000307210830_tess_v01_llc\hlsp_qlp_tess_ffi_s0005-0000000307210830_tess_v01_llc.fits: "Key 'PDCSAP_FLUX' does not exist."
Error in ../data\mastDownload\HLSP\hlsp_qlp_tess_ffi_s0008-0000000307210830_tess_v01_llc\hlsp_qlp_tess_ffi_s0008-0000000307210830_tess_v01_llc.fits: "Key 'PDCSAP_FLUX' does not exist."
Error in ../data\mastDownload\HLSP\hlsp_qlp_tess_ffi_s0009-0000000307210830_tess_v01_llc\hlsp_qlp_tess_ffi_s0009-0000000307210830_tess_v01_llc.fits: "Key 'PDCSAP_FLUX' does not exist."
Error in ../data\mastDownload\HLSP\hlsp_qlp_tess_ffi_s0010-0000000307210830_tess_v01_llc\hlsp_qlp_tess_ffi_s0010-0000000307210830_tess_v01_llc.fits: "Key 'PDCSAP_FLUX' does 

In [ ]:
df = pd.DataFrame(all_features, columns=[
    "local_noise", "width", "symmetry", "local_slope",
    "window_ratio", "snr", "best_power"
])
df["label"] = all_labels
df["source_file"] = all_sources  # Add this line
print(df.label.value_counts())

df.head()

print(f"Total extracted samples: {len(all_labels)}")


label
0    74488
1      418
Name: count, dtype: int64
Total extracted samples: 74906


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.utils import resample
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
# ---- STEP 0: Select new feature set ----
X = df[["local_noise", "width", "symmetry", "local_slope",
        "window_ratio", "snr", "best_power"]]
y = df["label"]

unique_files = df['source_file'].unique()
train_files, test_files = train_test_split(unique_files, test_size=0.3)

# Split data based on source files
train_mask = df['source_file'].isin(train_files)
X_train, y_train = X[train_mask], y[train_mask]
X_test, y_test = X[~train_mask], y[~train_mask]


In [ ]:

# ---- STEP 2: Balance only the TRAIN set (optional if imbalanced) ----
if y_train.value_counts().min() < 0.5 * y_train.value_counts().max():
    train_df = X_train.copy()
    train_df["label"] = y_train

    majority = train_df[train_df.label == train_df.label.value_counts().idxmax()]
    minority = train_df[train_df.label != train_df.label.value_counts().idxmax()]

    minority_upsampled = resample(
        minority,
        replace=True,
        n_samples=len(majority),
        random_state=42
    )

    train_df = pd.concat([majority, minority_upsampled])
    X_train, y_train = train_df.drop("label", axis=1), train_df["label"]

# ---- STEP 3: Pipeline (scaling + classifier) ----
pipe = Pipeline([
    ("scaler", StandardScaler()),  
    ("clf", RandomForestClassifier(
        n_estimators=100,
        max_depth=3,
        min_samples_leaf=10,
        min_samples_split=10,
        class_weight='balanced',  # Added class weight to handle imbalance
        random_state=42
    ))
])

# ---- STEP 4: Cross-validation on TRAIN set ----
scores = cross_val_score(pipe, X_train, y_train, cv=5)
print(f"Cross-validation accuracy: {scores.mean():.3f} ± {scores.std():.3f}")

# ---- STEP 5: Train final model ----
pipe.fit(X_train, y_train)

# ---- STEP 6: Evaluate on TEST set ----
y_pred = pipe.predict(X_test)
print(classification_report(y_test, y_pred))

Cross-validation accuracy: 0.998 ± 0.001
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23236
           1       0.67      1.00      0.80       124

    accuracy                           1.00     23360
   macro avg       0.83      1.00      0.90     23360
weighted avg       1.00      1.00      1.00     23360

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23236
           1       0.67      1.00      0.80       124

    accuracy                           1.00     23360
   macro avg       0.83      1.00      0.90     23360
weighted avg       1.00      1.00      1.00     23360



In [ ]:
# plt.figure(figsize=(10,5))
# plt.plot(time_clean[mask_window], flux_norm[mask_window], ".", markersize=2, label="Normalized")
# plt.plot(time_clean[mask_window], flux_smooth[mask_window], "r-", linewidth=1, label="Smoothed")
# plt.xlabel("Time (days)")
# plt.ylabel("Normalized Flux (PDCSAP_FLUX)")
# plt.title("Kepler Light Curve (Cleaned, Zoomed(120-130))")
# plt.legend()
# plt.show()


In [ ]:
# period = 2.5
# phase = (time_clean % period) / period

# plt.figure(figsize=(10,5))
# plt.plot(phase, flux_norm, ".", markersize=2, alpha=0.5)
# plt.xlabel("Phase (period = 2.5 days)")
# plt.ylabel("Normalized Flux")
# plt.title("Phase-Folded Light Curve")

In [ ]:
# threshold = np.percentile(flux_norm, 1)   # bottom 1%
# dip_mask = flux_norm < threshold

# plt.figure(figsize=(10,5))
# plt.plot(time_clean, flux_norm, ".", alpha=0.5, label="Flux")
# plt.plot(time_clean[dip_mask], flux_norm[dip_mask], "ro", label="Transit candidates")
# plt.xlabel("Time (days)")
# plt.ylabel("Normalized Flux")
# plt.title("Transit Detection (Threshold method)")
# plt.legend()
# plt.show()
